# Choose tutors
https://www.kaggle.com/c/choose-tutors

Задача в этом соревновании - предсказать, подойдет ли репетитор для подготовки к экзамену по математике или нет.

Можно использовать только следующие пакеты.

Pandas используется только для чтения и записи csv.

In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

#### Загружаем данные

In [69]:
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test.csv').set_index('Id')
TARGET = 'choose'

X = df_train.drop(TARGET, axis=1).to_numpy()
y = df_train[TARGET].to_numpy()
X_test = df_test.to_numpy()

#### Описание полей в датасетах

**Id** — индекс репетитора,
<br>**age** — возраст репетитора,
<br>**years_of_experience** — опыт репетитора в годах,
<br>**lesson_price** — текущая цена одного занятия,
<br>**qualification** — квалификация,
<br>**physics** — далее идут бинарные признаки того, что репетитор также преподает дополнительные предметы,
<br>**chemistry**,
<br>**biology**,
<br>**english**,
<br>**geography**,
<br>**history**,
<br>**mean_exam_points** — средняя оценка за экзамен у учеников,
<br>**choose** — целевой бинарный признак. Если значение `1`, то этот репетитор нам подходит.

#### Метрикой проверки является ROC AUC

In [70]:
def roc_auc_score(y_true, y_score):
    y_true_sorted = np.array([x for _, x in sorted(zip(y_score, y_true))])
    
    n = y_true_sorted.shape[0]
    n_pos = np.sum(y_true_sorted)
    n_neg = n - n_pos
    
    tp, fp = 0, 0
    roc = np.zeros((n + 1, 2))
    
    for i, class_ in enumerate(reversed(y_true_sorted)):
        if class_ == 1:
            tp += 1
        else:
            fp += 1
            
        roc[i + 1, 0] = fp / n_neg
        roc[i + 1, 1] = tp / n_pos
    
    score = 0
    for i in range(n):
        score += (roc[i + 1, 0] - roc[i, 0]) * roc[i, 1] 
    
    return score

#### Класс модели логистической регрессии

In [71]:
class LogisticRegression:
    def __init__(self, n_iter=1000, alpha=1e-4, threshold=.5, l1=0, l2=0, random_state=None):
        self.n_iter = n_iter
        self.alpha = alpha
        self.threshold = threshold
        self.l1 = l1
        self.l2 = l2
        self.random_state = random_state
        self.W = None

    @staticmethod
    def calc_logloss(y, y_pred, epsilon=1e-8):
        y = y.astype(np.float64)  # Привожу к типу вещественному,
        y_pred = y_pred.astype(np.float64)  # иначе ошибка в первом примере состоящем только из целых нудей и единиц

        y_pred[y_pred > 1 - epsilon] = 1 - epsilon  # Ограничим значения предсказанных значений интервалом [epsilon; 1 - epsilon]
        y_pred[y_pred < epsilon] = epsilon

        err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
        return err

    @staticmethod
    def sigmoid(z):
        res = 1 / (1 + np.exp(-z))
        return res

    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)

        self.W = np.random.randn(X.shape[1])
        n = X.shape[0]

        for i in range(1, self.n_iter + 1):
            z = self.W @ X.T
            y_pred = self.sigmoid(z)

            self.W -= self.alpha * (1 / n * np.dot((y_pred - y), X) + self.l1 * np.sign(self.W) + self.l2 * self.W)

    def predict_proba(self, X):
        z = np.dot(self.W, X.T)
        preds = self.sigmoid(z)
        return preds

    def predict(self, X):
        preds = (self.predict_proba(X) > self.threshold).astype('int')
        return preds

#### Классы для построения деревьев

In [72]:
class Node:
    def __init__(self, index, t, left_branch, right_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.left_branch = left_branch  # поддерево <= t
        self.right_branch = right_branch  # поддерево > t

    def classify(self, sample):
        if sample[self.index] <= self.t:
            return self.left_branch.classify(sample)
        else:
            return self.right_branch.classify(sample)

    def probability(self, sample):
        if sample[self.index] <= self.t:
            return self.left_branch.probability(sample)
        else:
            return self.right_branch.probability(sample)

In [73]:
class Leaf:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        regression = False

        # подсчет количества объектов разных классов
        classes = {0: 0, 1: 0}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label == 0 or label == 1:
                classes[label] += 1
            else:
                regression = True
                break

        if regression:
            self.class_ = np.mean(self.labels)
            if self.class_ < 0:
                self.class_ = 0
            self.proba = self.class_
        else:
            #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его
            self.class_ = max(classes, key=classes.get)
            self.proba = classes[1] / self.labels.shape[0]

    def classify(self, sample=None):
        return self.class_

    def probability(self, sample=None):
        return self.proba

In [74]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.tree = None

    @staticmethod
    def gini(labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2

        return impurity

    def quality(self, left_labels, right_labels, current_gini):
        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        return current_gini - p * self.gini(left_labels) - (1 - p) * self.gini(right_labels)

    @staticmethod
    def split(data, labels, index, t):
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)
        return data[left], data[right], labels[left], labels[right]

    def find_best_split(self, data, labels):
        current_gini = self.gini(labels)

        best_quality = 0
        best_t = None
        best_index = None
        
        feature_indexes = range(data.shape[1])
        if self.max_features == 'sqrt':
            n_indexes = int(np.sqrt(data.shape[1])) + 1
            feature_indexes = np.random.choice(feature_indexes, n_indexes, replace=False)
        
        for index in feature_indexes:
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < self.min_samples_leaf or len(false_data) < self.min_samples_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, current_gini)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index

    def build_tree(self, X, y, depth=0):
        quality, t, index = self.find_best_split(X, y)
        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества или сработал критерий останова
        if quality == 0 or self.max_depth and depth >= self.max_depth:
            return Leaf(X, y)

        true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)

        # Рекурсивно строим два поддерева
        left_branch = self.build_tree(true_data, true_labels, depth + 1)
        right_branch = self.build_tree(false_data, false_labels, depth + 1)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, left_branch, right_branch)

    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def predict(self, X):
        return np.array([self.tree.classify(sample) for sample in X])

    def predict_proba(self, X):
        return np.array([self.tree.probability(sample) for sample in X])

#### Класс случайного леса для решения задач классификации

In [75]:
class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_leaf=1, silent=True, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.silent = silent
        self.random_state = random_state
        self.forest = []

    def get_bootstrap(self, data, labels):
        n_samples = data.shape[0]
        bootstrap = []

        for i in range(self.n_estimators):
            b_data = np.zeros(data.shape)
            b_labels = np.zeros(labels.shape)

            if self.random_state:
                np.random.seed(self.random_state)

            for j in range(n_samples):
                sample_index = np.random.randint(0, n_samples - 1)
                b_data[j] = data[sample_index]
                b_labels[j] = labels[sample_index]
            bootstrap.append((b_data, b_labels))

        return bootstrap

    def fit(self, X, y):
        bootstrap = self.get_bootstrap(X, y)
        for i, (b_data, b_labels) in enumerate(bootstrap):
            tree = DecisionTree(self.max_depth, self.min_samples_leaf, max_features='sqrt')
            tree.fit(b_data, b_labels)
            self.forest.append(tree)
            if not self.silent and i % (self.n_estimators / 10) == 0:
                print(f'Построено деревьев {i}')

    def predict(self, X):
        preds = [tree.predict(X) for tree in self.forest]
        preds_per_object = list(zip(*preds))
        voted_predictions = np.array([max(set(obj), key=obj.count) for obj in preds_per_object])
        return voted_predictions

    def predict_proba(self, X):
        preds = [tree.predict_proba(X) for tree in self.forest]
        preds_per_object = list(zip(*preds))
        predictions = np.array([sum(obj) / len(self.forest) for obj in preds_per_object])
        return predictions

#### Класс градиентного бустинга на решающих деревьях

In [76]:
class GradientBoosting:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_leaf=1, learning_rate=.3, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.learning_rate = learning_rate
        self.trees = []
        self.coefs = [1] * n_estimators
        self.random_state = random_state

    def predict_proba(self, X):
        preds = np.zeros(X.shape[0])
        for i, x in enumerate(X):
            for tree, coef in zip(self.trees, self.coefs):
                preds[i] += self.learning_rate * coef * tree.predict_proba([x])[0]
        return preds

    def fit(self, X, y):
        tree = DecisionTree(self.max_depth, self.min_samples_leaf)
        tree.fit(X, y)
        preds = self.learning_rate * self.coefs[0] * tree.predict_proba(X)
        self.trees.append(tree)

        for i in range(1, self.n_estimators):
            tree = DecisionTree(self.max_depth, self.min_samples_leaf)
            tree.fit(X, y - preds)
            preds += self.learning_rate * self.coefs[i] * tree.predict_proba(X)
            self.trees.append(tree)

#### Масштабирование

In [77]:
class StandardScaler:
    def __init__(self):
        self.std = 0
        self.mean = 0
    
    def fit(self, X):
        self.std = np.std(X, axis=0)
        self.mean = np.mean(X, axis=0)
    
    def transform(self, X):
        return (X - self.mean) / self.std
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

#### Балансировка классов

In [78]:
class OverSampling:
    def __init__(self, sampling_strategy=1, algorithm='SMOTE', k_neighbors=5, random_state=None):
        self.sampling_strategy = sampling_strategy
        self.algorithm = algorithm
        self.k_neighbors = k_neighbors
        self.random_state = random_state

    @staticmethod
    def dist(x1, x2):
        distance = 0
        for i in range(len(x1)):
            distance += np.square(x1[i] - x2[i])
        return np.sqrt(distance)

    def fit_resample(self, X, y):
        need_samples = 0
        if self.random_state:
            np.random.seed(self.random_state)

        data = np.c_[X, y]
        classes = {0: np.sum(data[:, -1] == 0), 1: np.sum(data[:, -1] == 1)}
        minor_class = min(classes, key=classes.get)

        n_minor = classes[minor_class]
        n_major = data.shape[0] - n_minor

        if n_minor / n_major < self.sampling_strategy:
            need_samples = int(n_major * self.sampling_strategy) - n_minor

        if need_samples == 0:
            return X, y

        k = self.k_neighbors
        if self.algorithm == 'SMOTE':
            if n_minor - 1 < k:
                k = n_minor - 1

        minor_data = X[data[:, -1] == minor_class]
        if need_samples < n_minor:
            sample_indexes = np.random.choice(n_minor, need_samples, replace=False)
        else:
            sample_indexes = np.random.choice(n_minor, need_samples)

        for i in sample_indexes:
            current_sample = minor_data[sample_indexes[i]]
            
            if self.algorithm == 'SMOTE':
                # Генерируем новый признак на основе kNN
                distances = []
    
                for j, sample in enumerate(minor_data):
                    if j == sample_indexes[i]:
                        distances.append((np.inf, j))
                    else:
                        distance = self.dist(current_sample, sample)
                        distances.append((distance, j))
    
                vectors = []
                for j, d in enumerate(sorted(distances)[0:k]):
                    vectors.append((minor_data[d[1]] - current_sample) / k * np.random.random() / (j + 1))
    
                current_sample += np.sum(vectors, axis=0)
        
            current_sample = np.append(current_sample, minor_class)
            data = np.vstack((data, current_sample))
        return data[:, :-1], data[:, -1]

#### Добавление новых признаков

In [79]:
def add_new_features(df_train, df_test, y, corr_limit=.1):
    df_train_new = df_train.copy()
    df_test_new = df_test.copy()

    for col1 in range(df_train.shape[1]):
        for col2 in range(df_train.shape[1]):

            if col1 != col2:
                test_feature = df_train[:, col1] / (df_train[:, col2] + 1e-8)
                corr = np.corrcoef(y, test_feature)[0][1]
                if np.abs(corr) > corr_limit:
                    df_train_new = np.c_[df_train_new, test_feature]
                    df_test_new = np.c_[df_test_new, df_test[:, col1] / (df_test[:, col2] + 1e-8)]

            if col1 >= col2:
                test_feature = df_train[:, col1] * df_train[:, col2]
                corr = np.corrcoef(y, test_feature)[0][1]
                if np.abs(corr) > corr_limit:
                    df_train_new = np.c_[df_train_new, test_feature]
                    df_test_new = np.c_[df_test_new, df_test[:, col1] * df_test[:, col2]]

    return df_train_new, df_test_new

#### Валидация

Подбираем лучшую модель перебором параметров на ооснове ROC AUC метрики для тестовой выборки

Если `y_test` параметр не пуст, то функция возвращает значение метрики `ROC AUC` на тренировочной и тестовой выборках.

Если `y_test` параметр не пуст, то функция возвращает результаты предсказаний.

In [84]:
def fit_predict(model_class, params, X, y, X_test, y_test=None, sampling_algorithm=None, sampling_strategy=None,
                new_features=False, random_state=None):
    train_data, test_data, train_labels = X.copy(), X_test.copy(), y.copy()

    if new_features:
        train_data, test_data = add_new_features(train_data, test_data, y)

    scaler = StandardScaler()
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)

    if y_test is not None:
        train_data_orig = train_data.copy()

    if sampling_algorithm:
        oversampler = OverSampling(random_state=random_state, algorithm=sampling_algorithm, sampling_strategy=sampling_strategy)
        train_data, train_labels = oversampler.fit_resample(train_data, train_labels)

    model = model_class(**params)
    model.fit(train_data, train_labels)
    preds = model.predict_proba(test_data)

    if y_test is not None:
        train_score = roc_auc_score(y, model.predict_proba(train_data_orig))
        test_score = roc_auc_score(y_test, preds)
        return train_score, test_score

    return preds

In [81]:
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=2)

In [70]:
for alpha in [1e-3, 3e-3, .01, .03, .1, .3, 1, 3]:
    for n_iter in [100, 300, 1000, 3000, 10000, 30000]:
        y_pred = fit_predict(
            LogisticRegression,
            {'n_iter': n_iter, 'alpha': alpha},
            train_data, train_labels, test_data, y_test=test_labels,
            sampling_algorithm='Random',
            sampling_strategy=1,
            new_features=True,
            random_state=1
        )
        print(n_iter, alpha, y_pred)

100 0.001 (0.4002109475356384, 0.39888180715592375)
300 0.001 (0.43044430952373514, 0.42730292955826904)
1000 0.001 (0.5296566576562123, 0.5173653627672936)
3000 0.001 (0.6738213005121956, 0.6524183129660837)
10000 0.001 (0.8076493915493729, 0.8003986236553321)
30000 0.001 (0.8338043528000744, 0.8207240112610005)
100 0.003 (0.4304949728869325, 0.4273459548652894)
300 0.003 (0.5183158272346674, 0.5072602298481657)
1000 0.003 (0.6738576637325551, 0.6524438955810686)
3000 0.003 (0.8022038971566539, 0.7942867043661456)
10000 0.003 (0.8338029227857906, 0.8207251741071362)
30000 0.003 (0.8495684216891792, 0.8333269376795903)
100 0.01 (0.5300063982924796, 0.517674679839387)
300 0.01 (0.6740002565854252, 0.6525485517332803)
1000 0.01 (0.807675336094236, 0.8004242062703172)
3000 0.01 (0.8338023099225261, 0.8207216855687292)
10000 0.01 (0.8509563526954829, 0.8345258320454827)
30000 0.01 (0.8609335623535412, 0.8447332954245694)
100 0.03 (0.6743489757829174, 0.652904382650801)
300 0.03 (0.80230052

In [ ]:
%%time
for max_depth in [2, 3, 4]:
    for learning_rate in [1e-2, 3e-2, .1, .3, 1]:
        for n_estimators in [10, 30, 100, 300, 1000]:
            preds = fit_predict(
                GradientBoosting,
                {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_leaf': 30, 'learning_rate': learning_rate},
                train_data, train_labels, test_data, y_test=test_labels,
                sampling_algorithm='Random',
                sampling_strategy=1,
                new_features=True
            )
            print(max_depth, learning_rate, n_estimators, preds)

0.3 10 (0.8696478651112131, 0.8467008310861486)
0.3 30 (0.8761060139046142, 0.8530488081408433)
0.3 100 (0.8819625352600599, 0.8665110778537216)


In [ ]:
%%time
for max_depth in [6, 9, 12]:
    for n_estimators in [50, 100, 200, 400, 1000]:
        preds = fit_predict(
            RandomForestClassifier,
            {'n_estimators': n_estimators, 'max_depth': max_depth},
            train_data, train_labels, test_data, y_test=test_labels,
            sampling_algorithm='Random',
            sampling_strategy=1,
            new_features=True
        )
        print(sampling_strategy, n_estimators, preds)

#### Обучение выбранной модели и вывод результатов в файл

In [ ]:
# y_pred = fit_predict(
#     GradientBoosting,
#     {'n_estimators': 50, 'max_depth': 3, 'min_samples_leaf': 30, 'learning_rate': .3},
#     X, y, df_test,
#     sampling_algorithm='Random',
#     sampling_strategy=.3,
#     new_features=True
# )

In [85]:
y_pred = fit_predict(
    LogisticRegression,
    {'n_iter': 10000, 'alpha': .3, 'random_state': 1},
    X, y, X_test,
    sampling_algorithm='Random',
    sampling_strategy=1,
    new_features=True
)

Пробовал отправлять результаты, полученные с помощью лучших (среди проверенных) моделей Градиентного бустинга, Деревьев решений и Логистической регрессии, а также их комбинации.

Лучший результат получился на основе логистической регрессии.

In [86]:
pd.DataFrame({'Id': df_test.index, TARGET: y_pred}).to_csv('Batorov_predictions.csv', index=False)